In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler, LabelEncoder

import sys
import six
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_graphviz

import warnings
warnings.filterwarnings("ignore");

#### Process Data

In [3]:
df = pd.read_csv("archive/art_coverage_by_country_clean.csv")
df = df.dropna()

# df.head()
# df = df.drop(['Estimated number of people living with HIV_min',
#        'Estimated number of people living with HIV_median',
#        'Estimated number of people living with HIV_max', "Reported number of people receiving ART", "Estimated number of people living with HIV"], axis = 1);

df = df.drop(["Reported number of people receiving ART"], axis = 1);
df = df.reset_index();
df = df.drop(["index"], axis = 1);

s = "Estimated ART coverage among people living with HIV (%)";
for i in range(len(df[s])): 
       pos = df[s][i].find("[");
       df[s][i] = df[s][i][:pos];

s2: str = "Estimated number of people living with HIV"
for i in range(len(df[s2])): 
       pos = df[s2][i].find('[');
       df[s2][i] = df[s][i][:pos];

df.head()

Country Estimated number of people living with HIV  \
0  Afghanistan                                         13   
1      Algeria                                         81   
2       Angola                                         27   
3    Argentina                                         61   
4      Armenia                                         53   

  Estimated ART coverage among people living with HIV (%)  \
0                                                 13        
1                                                 81        
2                                                 27        
3                                                 61        
4                                                 53        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             3500.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          3000.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          4400.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               53.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               44.0             

   Estimated ART coverage among people living with HIV (%)_max  \
0                                               20.0             
1                                               86.0             
2                                               31.0             
3                                               67.0             
4                                               65.0             

              WHO Region  
0  Eastern Mediterranean  
1                 Africa  
2                 Africa  
3               Americas  
4                 Europe

In [4]:
for i in range(len(df["WHO Region"])): 
    if df["WHO Region"][i] != "Africa" and df["WHO Region"][i] != "Europe":
        df["WHO Region"][i] = "Other"

df.head()

Country Estimated number of people living with HIV  \
0  Afghanistan                                         13   
1      Algeria                                         81   
2       Angola                                         27   
3    Argentina                                         61   
4      Armenia                                         53   

  Estimated ART coverage among people living with HIV (%)  \
0                                                 13        
1                                                 81        
2                                                 27        
3                                                 61        
4                                                 53        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             3500.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          3000.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          4400.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               53.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               44.0             

   Estimated ART coverage among people living with HIV (%)_max WHO Region  
0                                               20.0                Other  
1                                               86.0               Africa  
2                                               31.0               Africa  
3                                               67.0                Other  
4                                               65.0               Europe

In [5]:
# label: list[str] = ['Eastern Mediterranean', 'Africa', 'Americas', 'Europe', 'Western Pacific', 'South-East Asia'];
label: list[str] = ['Africa', 'Europe', 'Other'];
cnt: list[int] = [i for i in range(len(label))];

mp = {};
for i in range(len(label)):
    mp[label[i]] = cnt[i];

df['WHO Region'] = df['WHO Region'].map(mp);

In [6]:
mp_country = {};
for i in df["Country"]: 
    if (i not in mp_country): mp_country[i] = 1;
    else: mp_country[i] += 1;


tmp = {};
for a, b in mp_country.items():
    tmp[a] = 1;

for i in range(len(df["Country"])):
    s: str = df["Country"][i];
    df['Country'][i] = float(df["WHO Region"][i] + tmp[s] * 0.01);
    tmp[s] += 1;

In [7]:
df = df.drop(["Country"], axis = 1);
df.head()

Estimated number of people living with HIV  \
0                                         13   
1                                         81   
2                                         27   
3                                         61   
4                                         53   

  Estimated ART coverage among people living with HIV (%)  \
0                                                 13        
1                                                 81        
2                                                 27        
3                                                 61        
4                                                 53        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             3500.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          3000.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          4400.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               53.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               44.0             

   Estimated ART coverage among people living with HIV (%)_max  WHO Region  
0                                               20.0                     2  
1                                               86.0                     0  
2                                               31.0                     0  
3                                               67.0                     2  
4                                               65.0                     1

#### Assign Data and target

In [8]:
target: list[int] = df['WHO Region'];
data = df.drop(['WHO Region'], axis = 1);

sc = StandardScaler();
data = sc.fit_transform(data);
# data = pp.scale(data);

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state=42);

#### Model

##### Decision Tree

In [10]:
Tree = DecisionTreeClassifier(random_state = 42, max_depth = 5, min_samples_split = 3);
Tree.fit(x_train, y_train);

In [11]:
pred = Tree.predict(x_test);

In [12]:
print(classification_report(y_test, pred));

              precision    recall  f1-score   support

           0       0.60      0.38      0.46         8
           1       0.67      0.25      0.36         8
           2       0.50      0.83      0.62        12

    accuracy                           0.54        28
   macro avg       0.59      0.49      0.48        28
weighted avg       0.58      0.54      0.50        28



##### Random Forest

In [13]:
rf = RandomForestClassifier(n_estimators = 5, random_state = 42);
rf.fit(x_train, y_train);

pred = rf.predict(x_test);

print(classification_report(y_test, pred));

              precision    recall  f1-score   support

           0       0.43      0.38      0.40         8
           1       0.80      0.50      0.62         8
           2       0.50      0.67      0.57        12

    accuracy                           0.54        28
   macro avg       0.58      0.51      0.53        28
weighted avg       0.57      0.54      0.54        28



In [14]:
# np.float = float

# estimator = Id3Estimator();
# estimator.fit(data, target);

# export_graphviz(estimator.tree_, 'tree.dot', data);

In [15]:
# !dot -Tpdf tree.dot -o tree.pdf

In [16]:
df1 = pd.read_csv("archive/art_coverage_by_country_clean.csv")
df2 = pd.read_csv("archive/art_pediatric_coverage_by_country_clean.csv")

df2 = df2.drop(["WHO Region"], axis = 1);

df_all = pd.merge(df1, df2, on = "Country");

df_all = df_all.dropna();

df_all = df_all.reset_index();
df_all = df_all.drop(["index", "Country"], axis = 1);


s = "Reported number of children receiving ART"
for i in range(len(df_all[s])): 
    Str = list(df_all[s][i].split())
    df_all[s][i] = float(Str[0]);

for i in range(len(df_all["WHO Region"])): 
    if df_all["WHO Region"][i] != "Africa" and df_all["WHO Region"][i] != "Europe":
        df_all["WHO Region"][i] = "Other"

# df.head()
df_all.head()

Reported number of people receiving ART  \
0                                    920.0   
1                                  12800.0   
2                                  88700.0   
3                                  85500.0   
4                                   3100.0   

  Estimated number of people living with HIV  \
0                           7200[4100–11000]   
1                         16000[15000–17000]   
2                      330000[290000–390000]   
3                      140000[130000–150000]   
4                            6000[5300–6700]   

  Estimated ART coverage among people living with HIV (%)  \
0                                           13[7–20]        
1                                          81[75–86]        
2                                          27[23–31]        
3                                          61[55–67]        
4                                          52[45–58]        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             6000.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          5300.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          6700.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               52.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               45.0             

   Estimated ART coverage among people living with HIV (%)_max WHO Region  \
0                                               20.0                Other   
1                                               86.0               Africa   
2                                               31.0               Africa   
3                                               67.0                Other   
4                                               58.0                Other   

  Reported number of children receiving ART  \
0                                      60.0   
1                                     770.0   
2                                    4800.0   
3                                    1700.0   
4                                      40.0   

  Estimated number of children needing ART based on WHO methods  \
0                                       500[500-530]              
1                                       500[500-520]              
2                                 38000[30000-47000]              
3                                    1800[1600-2100]              
4                                       200[200-500]              

  Estimated ART coverage among children (%)  \
0                                 17[10-26]   
1                                 95[95-

In [17]:
s2: str = "Estimated number of people living with HIV"
for i in range(len(df_all[s2])): 
       pos = df_all[s2][i].find('[');
       df_all[s2][i] = df_all[s2][i][:pos];

s2: str = "Estimated ART coverage among people living with HIV (%)";
for i in range(len(df_all[s2])): 
       pos = df_all[s2][i].find('[');
       df_all[s2][i] = df_all[s2][i][:pos];

s2: str = "Estimated number of children needing ART based on WHO methods"
for i in range(len(df_all[s2])): 
       pos = df_all[s2][i].find('[');
       df_all[s2][i] = df_all[s2][i][:pos];

s2: str = "Estimated ART coverage among children (%)"
for i in range(len(df_all[s2])): 
       pos = df_all[s2][i].find('[');
       df_all[s2][i] = df_all[s2][i][:pos];

for i in range(len(df_all["WHO Region"])): 
    if df_all["WHO Region"][i] != "Africa" and df_all["WHO Region"][i] != "Europe":
        df_all["WHO Region"][i] = "Other"
        
df_all.head()

Reported number of people receiving ART  \
0                                    920.0   
1                                  12800.0   
2                                  88700.0   
3                                  85500.0   
4                                   3100.0   

  Estimated number of people living with HIV  \
0                                       7200   
1                                      16000   
2                                     330000   
3                                     140000   
4                                       6000   

  Estimated ART coverage among people living with HIV (%)  \
0                                                 13        
1                                                 81        
2                                                 27        
3                                                 61        
4                                                 52        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             6000.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          5300.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          6700.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               52.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               45.0             

   Estimated ART coverage among people living with HIV (%)_max WHO Region  \
0                                               20.0                Other   
1                                               86.0               Africa   
2                                               31.0               Africa   
3                                               67.0                Other   
4                                               58.0                Other   

  Reported number of children receiving ART  \
0                                      60.0   
1                                     770.0   
2                                    4800.0   
3                                    1700.0   
4                                      40.0   

  Estimated number of children needing ART based on WHO methods  \
0                                                500              
1                                                500              
2                                              38000              
3                                               1800              
4                                                200              

  Estimated ART coverage among children (%)  \
0                                        17   
1                                       

In [18]:
# label = ['Eastern Mediterranean', 'Africa', 'Americas', 'Europe', 'Western Pacific', 'South-East Asia'];
# label: list[str] = ['Africa', 'Europe', 'Other'];
# cnt = [i for i in range(len(label))];

# mp = {};
# for i in range(len(label)):
#     mp[label[i]] = cnt[i];

# df_all['WHO Region'] = df_all['WHO Region'].map(mp);
s = "WHO Region";
le = LabelEncoder();
df_all[s] = le.fit_transform(df_all[s]);

In [22]:
df_all.head()

Reported number of people receiving ART  \
0                                    920.0   
1                                  12800.0   
2                                  88700.0   
3                                  85500.0   
4                                   3100.0   

  Estimated number of people living with HIV  \
0                                       7200   
1                                      16000   
2                                     330000   
3                                     140000   
4                                       6000   

  Estimated ART coverage among people living with HIV (%)  \
0                                                 13        
1                                                 81        
2                                                 27        
3                                                 61        
4                                                 52        

   Estimated number of people living with HIV_median  \
0                                             7200.0   
1                                            16000.0   
2                                           330000.0   
3                                           140000.0   
4                                             6000.0   

   Estimated number of people living with HIV_min  \
0                                          4100.0   
1                                         15000.0   
2                                        290000.0   
3                                        130000.0   
4                                          5300.0   

   Estimated number of people living with HIV_max  \
0                                         11000.0   
1                                         17000.0   
2                                        390000.0   
3                                        150000.0   
4                                          6700.0   

   Estimated ART coverage among people living with HIV (%)_median  \
0                                               13.0                
1                                               81.0                
2                                               27.0                
3                                               61.0                
4                                               52.0                

   Estimated ART coverage among people living with HIV (%)_min  \
0                                                7.0             
1                                               75.0             
2                                               23.0             
3                                               55.0             
4                                               45.0             

   Estimated ART coverage among people living with HIV (%)_max  WHO Region  \
0                                               20.0                     2   
1                                               86.0                     0   
2                                               31.0                     0   
3                                               67.0                     2   
4                                               58.0                     2   

  Reported number of children receiving ART  \
0                                      60.0   
1                                     770.0   
2                                    4800.0   
3                                    1700.0   
4                                      40.0   

  Estimated number of children needing ART based on WHO methods  \
0                                                500              
1                                                500              
2                                              38000              
3                                               1800              
4                                                200              

  Estimated ART coverage among children (%)  \
0                                        17   
1                                 

In [19]:
target = df_all['WHO Region'];
data = df_all.drop(['WHO Region'], axis = 1);

sc = StandardScaler();
data = sc.fit_transform(data);

In [34]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.4, random_state = 42);

In [35]:
tree = DecisionTreeClassifier(random_state = 42, max_depth = 5, min_samples_split = 3);
tree.fit(x_train, y_train);

pred = tree.predict(x_test);

print(classification_report(y_test, pred));

              precision    recall  f1-score   support

           0       0.64      0.82      0.72        17
           1       1.00      0.50      0.67         2
           2       0.80      0.63      0.71        19

    accuracy                           0.71        38
   macro avg       0.81      0.65      0.70        38
weighted avg       0.74      0.71      0.71        38



In [36]:
rng = range(1, 100);
dep = range(1, 10);

scores = []

mx: float = 0

for i in rng: 
    for k in dep: 
        rf = RandomForestClassifier(n_estimators = i, random_state = 42, max_depth = k);
        rf.fit(x_train, y_train)
        if (rf.score(x_test, y_test) >= mx): 
            mx = rf.score(x_test, y_test)
            print(f"i: {i}, k: {k}, score: {rf.score(x_test, y_test)}")
        scores.append(rf.score(x_test, y_test))

i: 1, k: 1, score: 0.8157894736842105
i: 1, k: 2, score: 0.8157894736842105
i: 1, k: 3, score: 0.8421052631578947
i: 8, k: 2, score: 0.8421052631578947
i: 9, k: 2, score: 0.8421052631578947
i: 10, k: 5, score: 0.8421052631578947
i: 11, k: 2, score: 0.868421052631579
i: 34, k: 4, score: 0.868421052631579
i: 35, k: 4, score: 0.868421052631579
i: 58, k: 7, score: 0.868421052631579


In [37]:
rf = RandomForestClassifier(n_estimators=58, max_depth=7, random_state=42)
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86        17
           1       1.00      1.00      1.00         2
           2       0.94      0.79      0.86        19

    accuracy                           0.87        38
   macro avg       0.91      0.91      0.91        38
weighted avg       0.88      0.87      0.87        38

